La empresa ADL se dedica a la venta de productos a nivel regional en los Estados Unidos,
con equipos de ventas que operan en diferentes estados. La compañía ha establecido un
sistema de comisiones para incentivar a sus equipos de ventas a preferir la venta de
productos de alto margen antes que los de bajo margen de utilidad.
Como Analista de Datos, le solicitan analizar información sobre las ventas realizadas por
los equipos de ventas de ADL, incluyendo detalles sobre los productos vendidos, los
clientes, las fechas de los pedidos y los plazos de entrega.
Para resolver este desafío deberás cargar datos desde Hojas de Excel del archivo
US_Regional_Sales_Data.xlsx. En base a esto, debe generar una visualización de datos
como la que se muestra en las hojas Reporte1 y Reporte2 del archivo reporte.xlsx.
Para cumplir con lo solicitado, puedes aplicar los siguientes pasos:
1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un
DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando
pd.ExcelFile(archivo).sheet_names)


In [7]:
import pandas as pd #FernandoSanchez

# Obtener los nombres de las hojas del archivo
archivo_excel = 'US_Regional_Sales_Data.xlsx'
hojas = pd.ExcelFile(archivo_excel).sheet_names #FernandoSanchez

# Cargar los datos de cada hoja en un DataFrame independiente
dataframes = {hoja: pd.read_excel(archivo_excel, sheet_name=hoja) for hoja in hojas} #FernandoSanchez

# Visualizar los primeros registros de cada DataFrame
for hoja, df in dataframes.items(): #FernandoSanchez
    print(f"\nPrimeros registros de la hoja '{hoja}':") #FernandoSanchez
    print(df.head())




Primeros registros de la hoja 'Sales Orders Sheet':
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  _StoreID  _ProductID   
0   2018-06-19          USD             6           15       259          12  \
1   2018-07-02          USD            14           20       196          27   
2   2018-07-01          USD            21           16       213          16   
3   2018-06-07          USD            28           48       107          23   
4   2018-06-26          USD            22           49      

2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado
en una nueva variable llamada df_base.


In [23]:
# Combina los DataFrames usando validación many-to-one
df_base = dataframes[hojas[0]]  #FernandoSanchez Inicializa df_base con el primer DataFrame
for hoja in hojas[1:]:
    if hoja == 'Customers Sheet':
        df_base = pd.merge(df_base, dataframes[hoja], how='left', on='_CustomerID') #FernandoSanchez
    elif hoja == 'Store Locations Sheet':
        df_base = pd.merge(df_base, dataframes[hoja], how='left', on='_StoreID')
    elif hoja == 'Products Sheet':#FernandoSanchez
        df_base = pd.merge(df_base, dataframes[hoja], how='left', on='_ProductID')
    else:
        df_base = pd.merge(df_base, dataframes[hoja], how='left', on='_SalesTeamID')

# Visualizar los primeros registros de df_base
print("\nPrimeros registros de df_base:")
print(df_base.head())#FernandoSanchez


Primeros registros de df_base:
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...  AreaCode   
0   2018-06-19          USD             6           15  ...       631  \
1   2018-07-02          USD            14           20  ...       913   
2   2018-07-01          USD            21           16  ...       734   
3   2018-06-07          USD            28           48  ...       203   
4   2018-06-26          USD            22           49  ...       203   

   Population  Household Income  Median In

3. Agrega las siguientes columnas a df_ base:
Columna Cálculo
ProcurementDays OrderDate-ProcuredDate
ShippingDays ShipDate-OrderDate
DeliveryDays DeliveryDate-ShipDate
CustomerDays ShippingDays+DeliveryDays
hint: Asegúrate de que las columnas sean de tipo datetime.


In [28]:
# Agrega las columnas solicitadas y asegúrate de que sean de tipo datetime
df_base['ProcurementDays'] = (df_base['OrderDate'] - df_base['ProcuredDate']).dt.days #FernandoSanchez
df_base['ShippingDays'] = (df_base['ShipDate'] - df_base['OrderDate']).dt.days
df_base['DeliveryDays'] = (df_base['DeliveryDate'] - df_base['ShipDate']).dt.days
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']

# Visualizar los primeros registros de df_base con las nuevas columnas
print("\nPrimeros registros de df_base con las nuevas columnas:")
print(df_base.head())#FernandoSanchez


Primeros registros de df_base con las nuevas columnas:
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...  Water Area   
0   2018-06-19          USD             6           15  ...   160302131  \
1   2018-07-02          USD            14           20  ...     1309517   
2   2018-07-01          USD            21           16  ...     2254528   
3   2018-06-07          USD            28           48  ...     3739487   
4   2018-06-26          USD            22           49  ...     1086045   

      

4. Agrega una nueva columna discreta al df_base, con el nombre CustomerDaysInterval
que clasifique los valores de la columna CustomerDays en los siguientes intervalos:
○ 0 to 15 days
○ 15 to 30 days
○ 30 to 45 days
○ 45 to 60 days
○ 60 to 75 days
○ 75 to 90 days
hint: Utiliza pd.cut con estas etiquetas de Intervalo. Puedes convertir previamente
la columna CustomerDays a valores numéricos con pd.to_numeric(). Considera que
en este caso, el resultado es entregado en formato 10**9 segundos, por lo que
debes dividir el resultado por este valor

In [29]:
# Agrega la nueva columna CustomerDaysInterval
#FernandoSanchez No es necesario convertir con esa division ya que ya estan entregados en valor numerico ya que arriba se puso solo dias
intervalos = [0, 15, 30, 45, 60, 75, 90]
etiquetas_intervalos = ['0 to 15 days', '15 to 30 days', '30 to 45 days', '45 to 60 days', '60 to 75 days', '75 to 90 days']

df_base['CustomerDaysInterval'] = pd.cut(df_base['CustomerDays'], bins=intervalos, labels=etiquetas_intervalos, right=False)

# Visualizar los primeros registros de df_base con la nueva columna
print("\nPrimeros registros de df_base con la nueva columna CustomerDaysInterval:")
print(df_base.head()) #FernandoSanchez


Primeros registros de df_base con la nueva columna CustomerDaysInterval:
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...  Water Area   
0   2018-06-19          USD             6           15  ...   160302131  \
1   2018-07-02          USD            14           20  ...     1309517   
2   2018-07-01          USD            21           16  ...     2254528   
3   2018-06-07          USD            28           48  ...     3739487   
4   2018-06-26          USD            22           49  ...     

5. Utiliza una tabla pivote para mostrar el conteo de órdenes agrupando por Sales
Team en las filas y CustomerDaysInterval en las columnas. Este reporte debe
cuadrar con los datos de la pestaña Reporte1 del archivo reportes.xlsx

In [32]:
# Utiliza una tabla pivote para el conteo de órdenes
tabla_pivote = pd.pivot_table(df_base, values='OrderNumber', index='Sales Team', columns='CustomerDaysInterval', aggfunc='count', fill_value=0) #FernandoSanchez

# Visualizar la tabla pivote
print("\nTabla pivote de conteo de órdenes:")
print(tabla_pivote) #FernandoSanchez


Tabla pivote de conteo de órdenes:
CustomerDaysInterval  0 to 15 days  15 to 30 days  30 to 45 days   
Sales Team                                                         
Adam Hernandez                  71            176             55  \
Anthony Berry                   69            176             53   
Anthony Torres                  70            160             58   
Carl Nguyen                     85            184             45   
Carlos Miller                   56            145             46   
Chris Armstrong                 86            152             54   
Donald Reynolds                 87            167             42   
Douglas Tucker                  76            148             53   
Frank Brown                     71            156             48   
George Lewis                    92            169             54   
Jerry Green                     81            162             53   
Joe Price                       80            129             57   
Jonathan Haw

6. Agrega la siguiente columna calculada a la base:
KPI Cálculo
GrossMargin OrderQuantity*(UnitPrice*(1-Discount Applied)-UnitCost)
Visión de Negocio: El margen bruto es un indicador financiero que muestra la diferencia entre
los ingresos generados por la venta de un producto o servicio y los costos directos asociados
con su producción o adquisición.


In [35]:
# Agrega la columna calculada GrossMargin
df_base['GrossMargin'] = df_base['Order Quantity'] * (df_base['Unit Price'] * (1 - df_base['Discount Applied']) - df_base['Unit Cost']) #FernandoSanchez

# Visualizar los primeros registros de df_base con la nueva columna
print("\nPrimeros registros de df_base con la nueva columna GrossMargin:")
print(df_base.head()) #FernandoSanchez


Primeros registros de df_base con la nueva columna GrossMargin:
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...         Time Zone   
0   2018-06-19          USD             6           15  ...  America/New York  \
1   2018-07-02          USD            14           20  ...   America/Chicago   
2   2018-07-01          USD            21           16  ...   America/Detroit   
3   2018-06-07          USD            28           48  ...  America/New York   
4   2018-06-26          USD            22  

7. En esta parte debes aplicar un porcentaje a la columna GrossMargin, pero ese
porcentaje dependerá del intervalo en que se encuentre GrossMargin.
KPI Cálculo
CommissionsPercentage % de comisión según la tabla de comisiones
hint: Utiliza pd.cut con labels numéricas para que la columna resultante sea de tipo float

In [36]:
# Definir los intervalos y porcentajes de comisión
intervalos_comision = [-float('inf'), 0, 10000, 20000, 30000, 40000, float('inf')] #FernandoSanchez los float('inf') que se pusieron es para mandar intervalos al infinito ya que asi cualquier numero posterior a 40000 será tomado en cuenta
porcentajes_comision = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06]

# Aplicar pd.cut para obtener las etiquetas numéricas
df_base['CommissionsPercentage'] = pd.cut(df_base['GrossMargin'], bins=intervalos_comision, labels=False, right=False) #FernandoSanchez

# Mapear las etiquetas numéricas a los porcentajes de comisión
df_base['CommissionsPercentage'] = df_base['CommissionsPercentage'].map({i: p for i, p in enumerate(porcentajes_comision)}) #FernandoSanchez

# Calcular las comisiones
df_base['Commissions'] = df_base['GrossMargin'] * df_base['CommissionsPercentage'] #FernandoSanchez

# Visualizar los primeros registros de df_base con las nuevas columnas
print("\nPrimeros registros de df_base con las nuevas columnas de comisión:")
print(df_base.head()) #FernandoSanchez



Primeros registros de df_base con las nuevas columnas de comisión:
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...      Sales Team   
0   2018-06-19          USD             6           15  ...  Joshua Bennett  \
1   2018-07-02          USD            14           20  ...     Paul Holmes   
2   2018-07-01          USD            21           16  ...   Samuel Fowler   
3   2018-06-07          USD            28           48  ...   Carlos Miller   
4   2018-06-26          USD            22         

8. Calcula el monto de la comisión, utilizando la Tabla del Anexo.
KPI Cálculo
CommissionsAmount GrossMargin*CommissionsPercentag

In [37]:
# Calcular el monto de la comisión
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['CommissionsPercentage'] #FernandoSanchez

# Visualizar los primeros registros de df_base con la nueva columna de monto de comisión
print("\nPrimeros registros de df_base con la nueva columna de monto de comisión:")
print(df_base.head()) #FernandoSanchez


Primeros registros de df_base con la nueva columna de monto de comisión:
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...     Region   
0   2018-06-19          USD             6           15  ...  Northeast  \
1   2018-07-02          USD            14           20  ...    Midwest   
2   2018-07-01          USD            21           16  ...    Midwest   
3   2018-06-07          USD            28           48  ...       West   
4   2018-06-26          USD            22           49  ...  Northeas

9. Calcula la comisión sobre el margen bruto.
KPI Cálculo
NetMargin GrossMargin-CommissionsAmount

In [38]:
# Calcular el margen neto
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount'] #FernandoSanchez

# Visualizar los primeros registros de df_base con la nueva columna de margen neto
print("\nPrimeros registros de df_base con la nueva columna de margen neto:")
print(df_base.head()) #FernandoSanchez



Primeros registros de df_base con la nueva columna de margen neto:
   OrderNumber Sales Channel WarehouseCode ProcuredDate  OrderDate   ShipDate   
0  SO - 000101      In-Store  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-14  \
1  SO - 000102        Online  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-22   
2  SO - 000103   Distributor  WARE-UHY1004   2017-12-31 2018-05-31 2018-06-21   
3  SO - 000104     Wholesale  WARE-NMK1003   2017-12-31 2018-05-31 2018-06-02   
4  SO - 000105   Distributor  WARE-NMK1003   2018-04-10 2018-05-31 2018-06-16   

  DeliveryDate CurrencyCode  _SalesTeamID  _CustomerID  ...  ProcurementDays   
0   2018-06-19          USD             6           15  ...              151  \
1   2018-07-02          USD            14           20  ...              151   
2   2018-07-01          USD            21           16  ...              151   
3   2018-06-07          USD            28           48  ...              151   
4   2018-06-26          USD            22    

10. Utilizando una tabla pivote, muestra la suma de los valores GrossMargin, NetMargin y
CommissionAmount agrupando por Sales Team en las filas. Este reporte debe cuadrar con
los datos de la pestaña Reporte2 del archivo reportes.xlsx
Anexo
Tabla de comisiones
Intervalo Comisión
GrossMargin entre $0 y $100 5%
GrossMargin entre $100 y $1000 10%
GrossMargin entre $1000 y $10000 15%
GrossMargin entre $10000 y $100000 20%

In [45]:
# Definir la tabla de comisiones
intervalos_comisiones = [
    (0, 100),
    (100, 1000),
    (1000, 10000),
    (10000, 100000)
]

porcentajes_comisiones = [0.05, 0.1, 0.15, 0.2]

# Crear una función para asignar comisiones basadas en intervalos de GrossMargin
def asignar_comision(gross_margin):
    for i, intervalo in enumerate(intervalos_comisiones):
        if intervalo[0] <= gross_margin < intervalo[1]: #FernandoSanchez
            return porcentajes_comisiones[i] 
    return 0.0  # Si no cae en ningún intervalo, comisión es 0

# Aplicar la función para asignar comisiones a cada fila
df_base['CommissionsPercentage'] = df_base['GrossMargin'].apply(asignar_comision) #FernandoSanchez

# Calcular las comisiones y el margen neto
df_base['CommissionsAmount'] = df_base['GrossMargin'] * df_base['CommissionsPercentage'] #FernandoSanchez
df_base['NetMargin'] = df_base['GrossMargin'] - df_base['CommissionsAmount'] #FernandoSanchez

# Utilizar una tabla pivote para mostrar la suma de los valores
tabla_pivote_reporte = pd.pivot_table(df_base, values=['GrossMargin', 'NetMargin', 'CommissionsAmount'], index='SalesTeam', aggfunc='sum') #FernandoSanchez

# Visualizar la tabla pivote del reporte
print("\nTabla pivote de suma de valores por SalesTeam:")
print(tabla_pivote_reporte)


Tabla pivote de suma de valores por SalesTeam:
                     CommissionsAmount  GrossMargin      NetMargin
Sales Team                                                        
Adam Hernandez           126637.207525  779095.1955  652457.987975
Anthony Berry            124061.181475  767943.9500  643882.768525
Anthony Torres           117962.211675  743901.4355  625939.223825
Carl Nguyen              123843.212050  774781.4675  650938.255450
Carlos Miller            110882.093875  668752.9290  557870.835125
Chris Armstrong          122565.707975  781608.7005  659042.992525
Donald Reynolds          143768.338700  885386.8095  741618.470800
Douglas Tucker           120806.403550  745528.5640  624722.160450
Frank Brown              104839.466050  670881.2510  566041.784950
George Lewis             136852.255325  839150.7460  702298.490675
Jerry Green              126660.436425  788587.7555  661927.319075
Joe Price                128822.930100  787707.4425  658884.512400
Jonathan Hawki